## Introduction
1. 데이터 검사
2. 메타데이터 정의
3. Descriptive statistics
4. 불균형 클래스 처리
5. 데이터 품질 검사
6. Exploratory Data 시각화
7. Feature engineering
8. Feature selection
9. Feature Scaling


In [7]:
#필요한 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns',100)
# column이 많은 데이터는 가운데 column을 생략하고 나타내는데 생략 없이 스크롤로 볼 수 있게 펼쳐주려면 이 함수를 활용한다.
# set_option 안에 최대 column수를 지정하면 된다

In [12]:
# 데이터를 로드 하자
train=pd.read_csv('C:/Users/LEEKYOUNGSEON/Kaggle2/week01/Kyungdun/porto-seguro-safe-driver-prediction/input/train.csv')
test=pd.read_csv('C:/Users/LEEKYOUNGSEON/Kaggle2/week01/Kyungdun/porto-seguro-safe-driver-prediction/input/test.csv')

### Data at first sight
- 다음은 대회의 데이터 설명문에서 발췌했다.
- 비슷한 그룹에 속하는 feature 이름(ind, reg, car,calc)에 태그가 지정된다.
- feature 이름에는 이진 feature를 나타내는 postfix bin과 범주형 feature를 나타내는 cat이 포함됩니다.
- 이러한 지정이 없는 feature는 연속형이거나 순서형이다
- -1은 feature가 관측치에 누락되었다는 뜻이다.
- target 열은 해당 정책 holder에게 클레임이 접수되었는지 여부를 나타낸다.
---
이 모든 것들을 확인하기 위해 첫번째 행과 마지막 행을 간단히 살펴보자.

In [13]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0


In [14]:
train.tail()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
595207,1488013,0,3,1,10,0,0,0,0,0,1,0,0,0,0,0,13,1,0,0,0.5,0.3,0.692820,10,1,-1,0,1,1,1,1,0,1,31,3,0.374166,0.684631,0.385487,2.645751,0.4,0.5,0.3,3,0,9,0,9,1,12,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,0.7,1.382027,9,1,-1,0,-1,15,0,0,2,1,63,2,0.387298,0.972145,-1.000000,3.605551,0.2,0.2,0.0,2,4,8,6,8,2,12,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,0,0,0,0,0,0,12,1,0,0,0.9,0.2,0.659071,7,1,-1,0,-1,1,1,1,2,1,31,3,0.397492,0.596373,0.398748,1.732051,0.4,0.0,0.3,3,2,7,4,8,0,10,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.9,0.4,0.698212,11,1,-1,0,-1,11,1,1,2,1,101,3,0.374166,0.764434,0.384968,3.162278,0.0,0.7,0.0,4,0,9,4,9,2,11,4,1,4,2,0,1,1,1,0,0
595211,1488027,0,0,1,8,0,0,1,0,0,0,0,0,0,0,0,7,1,0,0,0.1,0.2,-1.000000,7,0,-1,0,-1,0,1,0,2,1,34,2,0.400000,0.932649,0.378021,3.741657,0.4,0.0,0.5,2,3,10,4,10,2,5,4,4,3,8,0,1,0,0,0,0


우리는 다음을 확인할 수 있다.
- 이항 변수
- 범주 값이 정수인 범주형 변수
- 정수나 실수 값을 갖는 다른 변수
- 결측치를 나타내는 -1이 있는 변수
- target 변수, ID 변수
---
- train data에 있는 행과 열의 수를 살펴보자.

In [15]:
train.shape

(595212, 59)

- 59개의 변수와 595212개의 행이 있다.
- test data에 같은 수의 변수가 있는지 확인하자.
- train data에 중복된 행이 있는지 확인하자.

In [16]:
train.drop_duplicates()
#drop_duplicates는 중복된 행이 제거된 dataframe을 반환한다.
train.shape

(595212, 59)

- 중복된 행을 제거했는데도 아까와 같은 것을 보니 중복된 행은 없다.

In [17]:
test.shape

(892816, 58)

- test는 변수가 하나 적지만 그것은 target data라서 괜찮다.
- 이번에는 각 유형별로 얼마나 많은 변수가 있는지 알아보자.
- 나중에 범주형 변수에 대해 더미 변수를 만들 수 있다.
- 더미 변수란 male female이 있을 때 male 0 female 1로 변환된 변수이다.
- bin 변수는 이항변수이므로 더미화하지 않아도 된다.

In [20]:
train.info()
# train data의 변수 대한 정보를 나열.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              595212 non-null  int64  
 1   target          595212 non-null  int64  
 2   ps_ind_01       595212 non-null  int64  
 3   ps_ind_02_cat   595212 non-null  int64  
 4   ps_ind_03       595212 non-null  int64  
 5   ps_ind_04_cat   595212 non-null  int64  
 6   ps_ind_05_cat   595212 non-null  int64  
 7   ps_ind_06_bin   595212 non-null  int64  
 8   ps_ind_07_bin   595212 non-null  int64  
 9   ps_ind_08_bin   595212 non-null  int64  
 10  ps_ind_09_bin   595212 non-null  int64  
 11  ps_ind_10_bin   595212 non-null  int64  
 12  ps_ind_11_bin   595212 non-null  int64  
 13  ps_ind_12_bin   595212 non-null  int64  
 14  ps_ind_13_bin   595212 non-null  int64  
 15  ps_ind_14       595212 non-null  int64  
 16  ps_ind_15       595212 non-null  int64  
 17  ps_ind_16_

- 데이터형식이 모두 정수 혹은 실수라는 것을 알 수 있다.
- null 값은 -1로 대체되므로 non-null은 정상이다.

### Metadata
- 데이터 관리를 쉽게 하기위해 dataframe의 변수에 대한 메타정보를 저장하자.
- 분석, 시각화, 모델링을 위해 특정 변수를 선택하고 싶을 때 좋다.
    - role: input, ID, target
    - level: nominal, interval, ordinal, binary
    - keep: True, False
    - dtype: int, float, str # 정수 실수 글자


> - f는 train의 모든 행인데 굳이 train으로 표현하지 않고 train[f]로 표현한 이유가 뭘까?
> - 변수에 대한 모든 메타데이터를 포함한 dictionary 만들기<<이건 굳이 왜만들지?

In [66]:
data=[] #리스트로 지정
for f in train.columns:
    #role 지정
    if f == 'target':
        role='target'
    elif f == 'id':
        role='id'
    else:
        role='input'


    # level 지정
    if 'bin' in f or f=='target':
        level='binary'
    elif 'cat' in f or f=='id':
        level='nominal'
    elif train[f].dtype==int:
        level='ordinal'
    elif train[f].dtype==float:
        level='interval'
        
    # f는 train의 모든 행인데 굳이 train으로 표현하지 않고 train[f]로 표현한 이유가 뭘까?

    # id를 제외한 모든 변수는 keep을 true로 초기화한다.
    keep=True
    if f=='id':
        keep=False

    # data type 정의하기
    dtype=train[f].dtype


    # 변수에 대한 모든 메타데이터를 포함한 dictionary 만들기<<이건 굳이 왜만들지?
    f_dict={
        'varname':f,
        'role':role,
        'level':level,
        'keep':keep,
        'dtype':dtype
    }
    data.append(f_dict)
    # 목록 끝에 추가하기.

    meta=pd.DataFrame(data,columns=['varname','role','level','keep','dtype'])
    meta.set_index('varname',inplace=True)
    # 하나 이상의 기존 column이나 array를 사용하여 index를 세팅한다.


In [67]:
meta

,role,level,keep,dtype
varname,,,,
id,id,nominal,False,int64
target,target,binary,True,int64
ps_ind_01,input,binary,True,int64
ps_ind_02_cat,input,nominal,True,int64
ps_ind_03,input,nominal,True,int64
ps_ind_04_cat,input,nominal,True,int64
ps_ind_05_cat,input,nominal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


- 삭제되지 않은 모든 nominal 변수들을 추출하는 예시이다.
- 따라는 하겠는데 이걸 내가 스스로는 못할 것 같다 ..
- role, level 별 변수들의 수를 나타내보자.

In [58]:
pd.DataFrame({'count':meta.groupby(['role','level'])['role'].size()}).reset_index()
#reset_index 인덱스를 재설정하거나 인덱스의 level을 재설정한다.

,role,level,count
0,id,nominal,1
1,input,binary,20
2,input,interval,21
3,input,nominal,16
4,target,binary,1


### Descriptive statistics
- dataframe에 describe method를 이용할 수 있다. 
- 하지만 범주형 변수에 평균, 분산 등을 계산하는건 의미가 없다.
- 범주형 변수는 나중에 시각적으로 살펴보자.
- 메타파일을 이용해서 descriptive statistics를 계산하고자는 변수를 선택하자.
- 데이터 유형별로 이 작업을 수행한다.
---
#### Interval variables

In [50]:
v=meta[(meta.level=='interval')& (meta.keep)].index
train[v].describe()
#describe() 기술통계량을 생성합니다. 
#NaN값을 제외한 분포, 중심 경향, 분산을 요약하는 통계.

,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.610991,0.439184,0.551102,0.379945,0.813265,0.276256,3.065899,0.449756,0.449589,0.449849,2.372081,1.885886,7.689445,3.005823,9.225904,2.339034,8.433590,5.441382,1.441918,2.872288,7.539026
std,0.287643,0.404264,0.793506,0.058327,0.224588,0.357154,0.731366,0.287198,0.286893,0.287153,1.117219,1.134927,1.334312,1.414564,1.459672,1.246949,2.904597,2.332871,1.202963,1.694887,2.746652
min,0.000000,0.000000,-1.000000,-1.000000,0.250619,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.200000,0.525000,0.316228,0.670867,0.333167,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000
50%,0.700000,0.300000,0.720677,0.374166,0.765811,0.368782,3.316625,0.500000,0.400000,0.500000,2.000000,2.000000,8.000000,3.000000,9.000000,2.000000,8.000000,5.000000,1.000000,3.000000,7.000000
75%,0.900000,0.600000,1.000000,0.400000,0.906190,0.396485,3.605551,0.700000,0.700000,0.700000,3.000000,3.000000,9.000000,4.000000,10.000000,3.000000,10.000000,7.000000,2.000000,4.000000,9.000000
max,0.900000,1.800000,4.037945,1.264911,3.720626,0.636396,3.741657,0.900000,0.900000,0.900000,5.000000,6.000000,10.000000,9.000000,12.000000,7.000000,25.000000,19.000000,10.000000,13.000000,23.000000


#### reg variables
- 오직 ps_reg_03에만 결측치가 있다.
- 범위는 변수마다 다르다. StandardScaler로 스케일링을 할 수 있지만 사용하려는 Classifier에 따라 다르다.

#### car variables
- ps_car_12 and ps_car_14에 결측치가 있다.
> 이거 노트에는 car15라는데 14에 결측치가 있는게 맞지 않나?
- 범위가 달라서 스케일링 적용이 불가능하다.

#### calc variables
- 결측치 없다.
- 최댓값이 0.9 인 것들은 일종의 비율로 볼 수 있다.
- calc 123이 분포가 유사하다.

- 전체적으로 구간 변수의 범위가 작다. 데이터를 익명화하기 위해 로그 변환 같은 것이 적용되었을 수도 있다.
---
#### Ordinal variables (순서형 변수)

> #ordinal이 없어서 오류가 일어났다! 근데 ordinal이 왜 없는지 모르겠다 ㅠㅠ

In [49]:
v=meta[(meta.level=='ordinal')&(meta.keep)].index

train[v].describe()


ValueError: Cannot describe a DataFrame without columns

- 오직 하나의 결측치만 남았다. ps_car_11
- 다양한 범위를 다루기 위해 스케일링 할 수 있다.
---
#### Binary variables

In [68]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

,target,ps_ind_01,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.036448,1.900378,0.393742,0.257033,0.163921,0.185304,0.000373,0.001692,0.009439,0.000948,0.012451,7.299922,0.660823,0.121081,0.153446,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,0.187401,1.983789,0.488579,0.436998,0.370205,0.388544,0.019309,0.041097,0.096693,0.030768,0.127545,3.546042,0.473430,0.326222,0.360417,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,13.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- train data의 선형성은 매우 불균형적이다.
- 평균을 통해 대부분 변수가 0임을 짐작할 수 있다.